In [5]:
import pandas as pd
import sqlite3
import os
import glob

In [6]:
CSV_FOLDER = "../csv"  # CSVファイルがあるフォルダ
DB_PATH = "../db/pachislo.db"
HALL_ID = 101262
MODEL_NAME = "SマイジャグラーⅤKD"

In [7]:
# === テーブル作成 ===
def create_table(conn):
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS slot_data (
            date TEXT,
            hall_id INTEGER,
            model TEXT,
            unit_no INTEGER,
            start INTEGER,
            bb INTEGER,
            rb INTEGER,
            art INTEGER,
            max_medals INTEGER,
            bb_rate TEXT,
            rb_rate TEXT,
            art_rate TEXT,
            total_rate TEXT,
            last_start INTEGER,
            UNIQUE(date, unit_no)
        )
    """)
    conn.commit()

In [8]:
# === CSV → DB 挿入 ===
def insert_csv_to_db(csv_file, conn):
    date = os.path.basename(csv_file).split("_")[-1].replace(".csv", "")
    df = pd.read_csv(csv_file)

    # 必要な列名を統一
    df = df.rename(
        columns={
            "台番号": "unit_no",
            "累計スタート": "start",
            "BB回数": "bb",
            "RB回数": "rb",
            "ART回数": "art",
            "最大持玉": "max_medals",
            "BB確率": "bb_rate",
            "RB確率": "rb_rate",
            "ART確率": "art_rate",
            "合成確率": "total_rate",
            "前日最終スタート": "last_start",
        }
    )

    df["date"] = date
    df["hall_id"] = HALL_ID
    df["model"] = MODEL_NAME

    expected_columns = [
        "date",
        "hall_id",
        "model",
        "unit_no",
        "start",
        "bb",
        "rb",
        "art",
        "max_medals",
        "bb_rate",
        "rb_rate",
        "art_rate",
        "total_rate",
        "last_start",
    ]
    
    df = df[[col for col in expected_columns if col in df.columns]]
    
    cursor = conn.cursor()
    for _, row in df.iterrows():
        cursor.execute(
            """
            INSERT OR IGNORE INTO slot_data (
                date, hall_id, model, unit_no, start, bb, rb, art,
                max_medals, bb_rate, rb_rate, art_rate, total_rate, last_start
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """,
            tuple(row[col] for col in expected_columns),
        )
    conn.commit()

In [9]:
# === 実行 ===
def main():
    conn = sqlite3.connect(DB_PATH)
    create_table(conn)

    csv_files = glob.glob(os.path.join(CSV_FOLDER, "*.csv"))
    for file in csv_files:
        print(f"📥 インポート中: {file}")
        insert_csv_to_db(file, conn)

    conn.close()
    print("✅ 全CSVデータをDBに取り込みました！")

if __name__ == "__main__":
    main()

📥 インポート中: ../csv\EXA_FIRST_SﾏｲｼﾞｬｸﾞﾗｰⅤKD_2025-03-18.csv
📥 インポート中: ../csv\EXA_FIRST_SﾏｲｼﾞｬｸﾞﾗｰⅤKD_2025-03-19.csv
📥 インポート中: ../csv\EXA_FIRST_SﾏｲｼﾞｬｸﾞﾗｰⅤKD_2025-03-20.csv
📥 インポート中: ../csv\EXA_FIRST_SﾏｲｼﾞｬｸﾞﾗｰⅤKD_2025-03-21.csv
📥 インポート中: ../csv\EXA_FIRST_SﾏｲｼﾞｬｸﾞﾗｰⅤKD_2025-03-22.csv
📥 インポート中: ../csv\EXA_FIRST_SﾏｲｼﾞｬｸﾞﾗｰⅤKD_2025-03-23.csv
📥 インポート中: ../csv\EXA_FIRST_SﾏｲｼﾞｬｸﾞﾗｰⅤKD_2025-03-24.csv
📥 インポート中: ../csv\EXA_FIRST_SﾏｲｼﾞｬｸﾞﾗｰⅤKD_2025-03-25.csv
✅ 全CSVデータをDBに取り込みました！


In [10]:
# DB確認とJSON出力
df = pd.read_sql("SELECT * FROM slot_data", sqlite3.connect(DB_PATH))
df = df.drop_duplicates(subset=["unit_no", "date"])
df.to_json("C:/python/akmicWebApp/akmic-app_02/public/slot_data.json", orient="records", force_ascii=False)


In [11]:
df_pivot = df.drop_duplicates(subset=["unit_no", "date"])
df_pivot = df_pivot.pivot(index="unit_no", columns="date", values="total_rate")

In [12]:
df_pivot.shape

(81, 8)